In [ ]:
import pandas as pd
import numpy as np
import os
from pandas.tseries.offsets import *

# Station_V(V2)
功能 在"日期"目录下生成"线路-v.txt"文件，记录车辆编号、下一时刻时间、上下行、当前站、下一站、站间最大速度
运算逻辑 数据按车辆-线路group，按时间排序，相邻两行数据（i,j）中i的下一站等于（j的下一站+1）时记录<到站>

In [1]:
def get_max_v(bus1, line):
    # bus1[(bus1['O_SPEED'] == 0) & (bus1['O_LONGITUDE'] != 0.0) & (bus1['O_LATITUDE'] != 0.0)]
    len = bus1.shape[0]
    d = 0
    for i in range(len - 1):
        start_bus = bus1.iloc[i]
        station = start_bus['O_NEXTSTATIONNO']
        on = start_bus['O_UP']
        v = start_bus['O_SPEED']
        time = start_bus['O_TIME']
        car = start_bus['O_TERMINALNO']

        if i == 0:
            max_v = v
            
        next_bus = bus1.iloc[i + 1]
        next_station = next_bus['O_NEXTSTATIONNO']
        next_v = next_bus['O_SPEED']
        next_time = next_bus['O_TIME']
        next_car = next_bus['O_TERMINALNO']
        if (next_station == station):
            max_v = np.max([max_v, next_v])
        else:
            if (next_station == (station + 1)):
                
                max_v = np.max([max_v, next_v])

                ss = str(next_car) + ',' + str(next_time) + ',' + str(bus1['O_UP'].iloc[i + 1]) + ',' + str(
                    station - 1) + ',' + str(next_station - 1) + ',' + str(
                    max_v) + '\n'
                with open('./' + day + '/' + str(line) + '-v.txt', 'a+') as fw2:
                    fw2.write(ss)
                max_v = 0

In [ ]:
for d in range(25, 29):
    if d < 10:
        day = '0' + str(d)
    else:
        day = str(d)

    train = pd.read_csv("/home/b418-xiwei/xzl/notebook/DC/data/train201710"+day+".csv", sep=",")
    train['O_TIME'] = pd.to_datetime('2017-10-' + day + ' ' + train['O_TIME'], format='%Y-%m-%d %H:%M:%S')
    train = train.groupby(['O_LINENO', 'O_TERMINALNO']).apply(lambda x: x.sort_values(by='O_TIME', ascending=True))
    
    if not os.path.exists('./'+day+'/'):
        os.mkdir('./'+day+'/')
    line = train['O_LINENO'].drop_duplicates()
    print(day,line.shape)
    for i in range(line.shape[0]):
        l = line.iloc[i]
        buses = train[train['O_LINENO'] == l]
#         print(l, buses.shape)
        get_max_v(buses, l)
    print(day, "pre finished")

25 (543,)
